In [0]:
%sql
-- Create SQL Procedure for sp_ADS_SMA_CPP_MAIN_PREPARE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CPP_THECASE_PREPARE(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for sp_ADS_SMA_CPP_THECASE_PREPARE'
AS
BEGIN
    -- Declare local variables
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_SMA_PROC',
            step_name => 'ADS_SMA_CPP_THECASE_PREPARE',
            psource_sys_origin  => 'MAIN',
            process_key => p_process_key,
            target_table => 'STG_SMA_CASE_PHASE_PROPERTIES_THECASE',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

        TRUNCATE TABLE gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_THECASE;
        
        /* Fill Stage Table */
        INSERT INTO GAP_CATALOG.ADS_ETL_OWNER.STG_SMA_CASE_PHASE_PROPERTIES_THECASE
        SELECT 
            cidla, 
            get_json_object(JSONDATA, '$.theCase.casePhase') AS casephase, 
            timestamp  
        FROM (select cidla, type, timestamp, JSONDATA from gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full s where s.SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE) AND get_json_object(JSONDATA, '$.theCase.casePhase') IS NOT NULL 
        AND s.type IN (
            'CASE_CREATED',
            'CASE_COMPLETED',
            'CASE_WAITING',
            'CASE_RUNNING',
            'CASE_UPDATED',
            'CASE_PHASE_CHANGED',
            'CASE_RECONCILE',
            'CASE_TRANSITIONS_EVALUATED'
        ) 
        ) s
        LEFT JOIN (select case_key,case_source_id,cstat_key from  gap_catalog.ads_src_20250901.cases c
        WHERE c.case_valid_to = DATE'3000-01-01'
            AND c.case_source_system_id = 'SMA' 
            AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS') c
            ON c.case_source_id = s.cidla 
            AND s.type = 'CASE_TRANSITIONS_EVALUATED'
        LEFT JOIN gap_catalog.ads_src_20250901.case_status cs 
            ON c.cstat_key = cs.cstat_key 
            AND cs.cstat_valid_to = DATE'3000-01-01'
        WHERE (c.case_key IS NULL OR cs.CSTAT_SOURCE_ID = get_json_object(jsondata, '$.theCase.caseStatus')) ;

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

END;
    
END;